# 201: Exampville Mode Choice

Welcome to Exampville, the best simulated town in this here part of the internet!

Exampville is a demonstration provided with Larch that walks through some of the 
data and tools that a transportation planner might use when building a travel model. 

In [1]:
import os
import numpy as np
import pandas as pd 
import larch.numba as lx
from larch import P, X

/Users/jeffnewman/LocalGit/sharrow_pro/sharrow_pro/announce.py:4: ExtremelyDangerousMission: 

  good_news_everyone( ### sharrow advanced features are available ### )
/Users/jeffnewman/LocalGit/larch/larch/numba/model.py:20: UserWarning: 

### larch.numba is experimental, and not feature-complete ###
 the first time you import on a new system, this package will
 compile optimized binaries for your machine, which may take 
 a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )


In [2]:
lx.__version__

'5.5.9'

In this example notebook, we will walk through the creation of a tour mode choice model.

To begin, we'll re-load the tours and skims data from the 
data setup example.

In [3]:
hh, pp, tour, skims = lx.example(200, ['hh', 'pp', 'tour', 'skims'])

## Preprocessing

The Exampville data output contains a set of files similar to what we might
find for a real travel survey: network skims, and tables of households, persons,
and tours.  We'll need to connect these tables together to create a composite dataset
for mode choice model estimation.

We can merge data from other tables using the usual pandas syntax for merging.


In [4]:
from addicty import Dict

Mode = Dict(
    DA = 1,
    SR = 2,
    Walk = 3,
    Bike = 4,
    Transit = 5,
).freeze()

In [5]:
tour

,TOURID,HHID,PERSONID,DTAZ,TOURMODE,TOURPURP,N_STOPS,N_TRIPS,N_TRIPS_HBW,N_TRIPS_HBO,N_TRIPS_NHB
0,0,50000,60000,22,1,1,0,2,2,0,0
1,1,50000,60001,4,1,1,0,2,2,0,0
2,2,50000,60001,10,2,2,0,2,0,2,0
3,3,50000,60002,20,1,1,0,2,2,0,0
4,4,50000,60002,20,1,2,1,3,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
20734,20734,54998,72343,40,1,1,0,2,2,0,0
20735,20735,54998,72344,36,5,2,1,3,0,2,1
20736,20736,54999,72345,5,1,1,3,5,1,1,3
20737,20737,54999,72346,28,1,2,4,6,0,2,4


In [6]:
tour_dataset = lx.Dataset.from_idco(tour.set_index('TOURID'), alts=Mode)

In [7]:
dt = lx.DataTree(tour=tour_dataset)

In [8]:
dt.add_dataset('hh', hh.set_index('HHID'), relationships="tours.HHID @ hh.HHID")

In [9]:
dt.add_dataset('person', pp.set_index('PERSONID'), relationships="tours.PERSONID @ person.PERSONID")

In [10]:
od_skims = lx.Dataset.from_omx(skims)
dt.add_dataset(
    'od', od_skims, 
    relationships=(
        "hh.HOMETAZ @ od.otaz",
        "tours.DTAZ @ od.dtaz",
    ),
)
dt.add_dataset(
    'do', od_skims, 
    relationships=(
        "hh.HOMETAZ @ do.dtaz",
        "tours.DTAZ @ do.otaz",
    ),
)

In Exampville, there are only two kinds of trips: 

- work (purpose=1) and 
- non-work (purpose=2). 

We want to estimate a mode choice model for work trips, 
so we’ll begin by excluding all the other trips:

In [11]:
dt_work = dt.query_cases("TOURPURP == 1")

## Model Definition

And then we are ready to create our model.

In [12]:
m = lx.Model(datatree = dt_work)
m.title = "Exampville Work Tour Mode Choice v1"

We will explicitly define the set of utility functions 
we want to use.  Because the DataFrames we are using to 
serve data to this model contains exclusively `idco` format
data, we'll use only the `utility_co` mapping to define
a unique utility function for each alternative.

In [13]:
m.utility_co[Mode.DA] = (
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * X.AUTO_COST # dollars per mile
)

m.utility_co[Mode.SR] = (
        + P.ASC_SR
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * (X.AUTO_COST * 0.5) # dollars per mile, half share
        + P("LogIncome:SR") * X("log(INCOME)")
)

m.utility_co[Mode.Walk] = (
        + P.ASC_Walk
        + P.NonMotorTime * X.WALK_TIME
        + P("LogIncome:Walk") * X("log(INCOME)")
)

m.utility_co[Mode.Bike] = (
        + P.ASC_Bike
        + P.NonMotorTime * X.BIKE_TIME
        + P("LogIncome:Bike") * X("log(INCOME)")
)

m.utility_co[Mode.Transit] = (
        + P.ASC_Transit
        + P.InVehTime * X.TRANSIT_IVTT
        + P.OutVehTime * X.TRANSIT_OVTT
        + P.Cost * X.TRANSIT_FARE
        + P("LogIncome:Transit") * X('log(INCOME)')
)

To write a nested logit mode, we'll attach some nesting nodes to the 
model's `graph`.  Each `new_node` allows us to define the set of 
codes for the child nodes (elemental alternatives, or lower level nests)
as well as giving the new nest a name and assigning a logsum parameter.
The return value of this method is the node code for the newly created 
nest, which then can potenially be used as a child code when creating
a higher level nest.  We do this below, adding the 'Car' nest into the 
'Motor' nest.

In [14]:
dt_work.root_dataset

<larch.Dataset>
Dimensions:      (_0_caseid_: 7564, _1_altid_: 5)
Coordinates:
  * _0_caseid_   (_0_caseid_) int64 0 1 3 7 10 ... 20728 20729 20731 20734 20736
  * _1_altid_    (_1_altid_) int64 1 2 3 4 5
    alt_names    (_1_altid_) <U7 'DA' 'SR' 'Walk' 'Bike' 'Transit'
Data variables:
    HHID         (_0_caseid_) int64 50000 50000 50000 ... 54997 54998 54999
    PERSONID     (_0_caseid_) int64 60000 60001 60002 ... 72342 72343 72345
    DTAZ         (_0_caseid_) int64 22 4 20 25 25 4 19 22 ... 25 3 2 40 1 40 5
    TOURMODE     (_0_caseid_) int64 1 1 1 1 1 1 1 1 1 1 ... 2 1 5 5 2 1 1 1 1 1
    TOURPURP     (_0_caseid_) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    N_STOPS      (_0_caseid_) int64 0 0 0 2 1 0 0 0 0 0 ... 0 0 0 0 0 2 0 0 0 3
    N_TRIPS      (_0_caseid_) int64 2 2 2 4 3 2 2 2 2 2 ... 2 2 2 2 2 4 2 2 2 5
    N_TRIPS_HBW  (_0_caseid_) int64 2 2 2 1 1 2 2 2 2 2 ... 2 2 2 2 2 1 2 2 2 1
    N_TRIPS_HBO  (_0_caseid_) int64 0 0 0 1 1 0 0 0 0 0 ... 0 0 0 0 0 1 0 0 0 1
    N_TRIPS_NHB  (_0_caseid_) int64 0 0 0 2 1 0 0 0 0 0 ... 0 0 0 0 0 2 0 0 0 3

In [15]:
Car = m.graph.new_node(parameter='Mu:Car', children=[Mode.DA, Mode.SR], name='Car')
NonMotor = m.graph.new_node(parameter='Mu:NonMotor', children=[Mode.Walk, Mode.Bike], name='NonMotor')
Motor = m.graph.new_node(parameter='Mu:Motor', children=[Car, Mode.Transit], name='Motor')

Let's visually check on the nesting structure.

In [16]:
m.graph

The tour mode choice model's choice variable is indicated by 
the code value in 'TOURMODE', and this can be 
defined for the model using `choice_co_code`.

In [17]:
m.choice_co_code = 'TOURMODE'

We can also give a dictionary of availability conditions based 
on values in the `idco` data, using the `availability_co_vars`
attribute.  Alternatives that are always available can be indicated
by setting the criterion to 1.

In [18]:
m.availability_co_vars = {
    Mode.DA: 'AGE >= 16',
    Mode.SR: 1,
    Mode.Walk: 'WALK_TIME < 60',
    Mode.Bike: 'BIKE_TIME < 60',
    Mode.Transit: 'TRANSIT_FARE>0',
}

Then let's prepare this data for estimation.  Even though the
data is already in memory, the `load_data` method is used to 
pre-process the data, extracting the required values, pre-computing 
the values of fixed expressions, and assembling the results into
contiguous arrays suitable for computing the log likelihood values
efficiently.

## Model Estimation

In [20]:
m.loglike()

-10644.65822327678

In [28]:
m.data_as_loaded

<larch.Dataset>
Dimensions:     (_0_caseid_: 7564, _1_altid_: 5, _var_co: 8)
Coordinates:
  * _0_caseid_  (_0_caseid_) int64 0 1 3 7 10 ... 20728 20729 20731 20734 20736
    alt_names   (_1_altid_) <U7 'DA' 'SR' 'Walk' 'Bike' 'Transit'
  * _1_altid_   (_1_altid_) int64 1 2 3 4 5
  * _var_co     (_var_co) <U12 'AUTO_COST' 'AUTO_TIME' ... 'log(INCOME)'
Data variables:
    co          (_0_caseid_, _var_co) float64 0.5885 5.044 8.407 ... 34.77 10.05
    ch          (_0_caseid_, _1_altid_) float64 1.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    av          (_0_caseid_, _1_altid_) int8 1 1 1 1 0 1 1 0 ... 0 1 1 1 1 1 1 1

We can check on some important statistics of this loaded data even
before we estimate the model.

In [29]:
# m.dataframes.choice_avail_summary()

In [30]:
# m.dataframes.data_co.statistics()

If we are satisfied with the statistics we see above, we
can go ahead and estimate the model.

In [31]:
result = m.maximize_loglike(method='slsqp')

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
ASC_Bike,-0.258042,0.0,0.0,-inf,inf,0,,-0.258042
ASC_SR,1.422964,0.0,0.0,-inf,inf,0,,1.422964
ASC_Transit,6.754608,0.0,0.0,-inf,inf,0,,6.754608
ASC_Walk,8.621563,0.0,0.0,-inf,inf,0,,8.621563
Cost,-0.175704,0.0,0.0,-inf,inf,0,,-0.175704
InVehTime,-0.123720,0.0,0.0,-inf,inf,0,,-0.123720
LogIncome:Bike,-0.196979,0.0,0.0,-inf,inf,0,,-0.196979
LogIncome:SR,-0.193819,0.0,0.0,-inf,inf,0,,-0.193819
LogIncome:Transit,-0.557162,0.0,0.0,-inf,inf,0,,-0.557162
LogIncome:Walk,-0.522791,0.0,0.0,-inf,inf,0,,-0.522791


/Users/jeffnewman/LocalGit/larch/larch/model/optimization.py:307: UserWarning: slsqp may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn( # infinite bounds # )


After we find the best fitting parameters, we can compute
some variance-covariance statistics, incuding standard errors of
the estimates and t statistics, using `calculate_parameter_covariance`.

In [32]:
m.calculate_parameter_covariance()

Then we can review the results in a variety of report tables.

In [33]:
m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
ASC_Bike,-0.258,1.34,-0.19,,0.00
ASC_SR,1.42,1.00,1.42,,0.00
ASC_Transit,6.75,2.06,3.27,**,0.00
ASC_Walk,8.62,1.14,7.57,***,0.00
Cost,-0.176,0.120,-1.47,,0.00
InVehTime,-0.124,0.0292,-4.24,***,0.00
LogIncome:Bike,-0.197,0.124,-1.59,,0.00
LogIncome:SR,-0.194,0.135,-1.43,,0.00
LogIncome:Transit,-0.557,0.169,-3.29,***,0.00
LogIncome:Walk,-0.523,0.100,-5.21,***,0.00


In [34]:
m.estimation_statistics()

<xmle.Elem 'div' with 1 children>

## Save and Report Model

In [35]:
report = lx.Reporter(title=m.title)

In [47]:
report.append('# Parameter Summary')
report.append(m.parameter_summary())
report

,Value,Std Err,t Stat,Signif,Null Value
ASC_Bike,-0.258,1.34,-0.19,,0.00
ASC_SR,1.42,1.00,1.42,,0.00
ASC_Transit,6.75,2.06,3.27,**,0.00
ASC_Walk,8.62,1.14,7.57,***,0.00
Cost,-0.176,0.120,-1.47,,0.00
InVehTime,-0.124,0.0292,-4.24,***,0.00
LogIncome:Bike,-0.197,0.124,-1.59,,0.00
LogIncome:SR,-0.194,0.135,-1.43,,0.00
LogIncome:Transit,-0.557,0.169,-3.29,***,0.00
LogIncome:Walk,-0.523,0.100,-5.21,***,0.00


In [48]:
report << "# Estimation Statistics" << m.estimation_statistics()

<xmle.Elem 'div' with 16 children>

In [49]:
report << "# Utility Functions" << m.utility_functions()

alt,formula
1,+ P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST
2,+ P.ASC_SR + P.InVehTime * X.AUTO_TIME + P.Cost * 0.5 * X.AUTO_COST + P('LogIncome:SR') * X('log(INCOME)')
3,+ P.ASC_Walk + P.NonMotorTime * X.WALK_TIME + P('LogIncome:Walk') * X('log(INCOME)')
4,+ P.ASC_Bike + P.NonMotorTime * X.BIKE_TIME + P('LogIncome:Bike') * X('log(INCOME)')
5,+ P.ASC_Transit + P.InVehTime * X.TRANSIT_IVTT + P.OutVehTime * X.TRANSIT_OVTT + P.Cost * X.TRANSIT_FARE + P('LogIncome:Transit') * X('log(INCOME)')


In [50]:
report.save(
    '/tmp/exampville_mode_choice.html', 
    overwrite=True, 
    metadata=m,
)

'/tmp/exampville_mode_choice.html'